In [1]:
import pandas as pd
from sklearn.metrics import cohen_kappa_score
import numpy as np
from math import sqrt
import random
from pathlib import Path
import os
from statsmodels.stats.inter_rater import fleiss_kappa, aggregate_raters
import datetime
import json 

## Function definitions

In [2]:
def bootstrap_cqk(y_true, y_pred, quad=False, num_resamples=1000):
    """
    Bootstrap function for Cohen's Quadratic Kappa Score.

    Parameters
    ----------
    y_true: array-like
        True labels.
    y_pred: array-like
        Predicted labels.
    quad: bool, optional (default=False)
        If True, use quadratic weighting; otherwise, use linear weighting.
    num_resamples: int, optional (default=1000)
        Number of bootstrap resamples.

    Returns
    -------
    - float
        Mean of the kappa across bootstrap samples.
    - float
        Standard error of kappa (standard deviation of the sampling distribution).
    - float
        95% CI lower bound (2.5th percentile of the sorted bootstrap distribution).
    - float
        95% CI upper bound (97.5th percentile of the sorted bootstrap distribution).
    """
    # Combine true and predicted labels
    Y = np.array([y_true, y_pred]).T

    # List to store weighted kappas for each bootstrap sample
    weighted_kappas = []
    
    # Bootstrap resampling
    for i in range(num_resamples):
        # Randomly sample from the combined true and predicted labels
        Y_resample = np.array(random.choices(Y, k=len(Y)))
        y_true_resample = Y_resample[:, 0]
        y_pred_resample = Y_resample[:, 1]
        
        # Calculate Cohen's Kappa Score based on weighting option
        if not quad:
            weighted_kappa = cohen_kappa_score(y_true_resample.astype(str), y_pred_resample.astype(str))
        else:
            weighted_kappa = cohen_kappa_score(y_true_resample.astype(str), y_pred_resample.astype(str), weights='quadratic')
        
        # Append the calculated kappa score to the list
        weighted_kappas.append(weighted_kappa)

    # Calculate mean, standard error, and confidence intervals of the kappa scores
    return np.mean(weighted_kappas), np.std(weighted_kappas), np.percentile(weighted_kappas, 2.5), np.percentile(weighted_kappas, 97.5)


In [3]:
def create_confusion_matrix(list_1: list, list_2: list) -> pd.DataFrame:
    """
    Create a confusion matrix for comparing two lists of attributes.

    Parameters
    ----------
    list_1: list
        First list of attributes.
    list_2: list
        Second list of attributes.

    Returns
    -------
    The confusion matrix showing the counts of matching attributes. (pd.DataFrame)

    Note
    ----
    - If the lengths of list_1 and list_2 are not equal, a message is printed indicating potential mismatch.
    - The confusion matrix includes row and column totals, providing a comprehensive summary.

    Example
    -------
    - create_confusion_matrix(['A', 'B', 'A', 'C'], ['B', 'B', 'A', 'C'])
    """

    if not len(list_1) == len(list_2):
        print("Reviewer 1 and 2 may not have rated the same list of subjects.")
    else:
        list_attributes = sorted(list(set(list_1) | set(list_2)))

        size = len(list_attributes)
        matrix = pd.DataFrame(np.zeros((size + 1, size + 1)))

        # Populate the confusion matrix
        for k in range(size):
            for l in range(size):
                att_1 = list_attributes[k]
                att_2 = list_attributes[l]
                for i in range(len(list_1)):
                    if list_1[i] == att_1 and list_2[i] == att_2:
                        matrix.loc[k, l] += 1

        # Calculate row and column totals
        for i in range(size):
            for j in range(size):
                matrix.loc[size, i] += matrix.loc[j, i]
                matrix.loc[i, size] += matrix.loc[i, j]

        # Calculate the overall total
        for i in range(size):
            matrix.loc[size, size] += matrix.loc[i, size]

        # Normalize the matrix by dividing by the total number of ratings
        matrix = matrix / len(list_1)

    return matrix


In [4]:
def expected_proportion(matrix: pd.DataFrame):
    """
    Calculate the overall proportion of agreement expected by chance from the confusion matrix..

    Parameters
    ----------
    matrix: pd.DataFrame
        Confusion matrix representing the counts of matching attributes.

    Returns
    -------
    The overall proportion of agreement expected by chance (expected agreement).

    Example
    -------
    - expected_proportion(create_confusion_matrix(['A', 'B', 'A', 'C'], ['B', 'B', 'A', 'C']))
    """
    pe = 0
    k = len(matrix) - 1
    for i in range(k):
        pe += matrix.loc[i, k] * matrix.loc[k, i]
    return pe


def observed_proportion(matrix: pd.DataFrame):
    """
    Calculate the overall proportion of observed agreement from the confusion matrix.

    Parameters
    ----------
    matrix: pd.DataFrame
        Confusion matrix representing the counts of matching attributes.

    Returns
    -------
    The overall proportion of observed agreement.

    Example
    -------
    - observed_proportion(create_confusion_matrix(['A', 'B', 'A', 'C'], ['B', 'B', 'A', 'C']))
    """
    po = 0
    k = len(matrix) - 1
    for i in range(k):
        po += matrix.loc[i, i]
    return po


In [5]:
def sd_cohen(po, pe):
    """
    Cohen standard deviation.
    """
    sd_= sqrt((po*(1-po))/((1-pe)*(1-pe)))
    return sd_

In [6]:
def kappa(po, pe):
    """
    Calculate the kappa cohen score.
    """
    return (po-pe)/(1-pe)

In [7]:
# These functions provide a convenient way to organize and store Cohen's Kappa statistics in a DataFrame for further analysis or reporting

def generic_write_stat(df_final, category, method, kappa_, low_, high_, se_):
    df_final.loc[category, ("kappa score", method)]=kappa_
    df_final.loc[category, ("ci low", method)]=low_
    df_final.loc[category, ("ci high", method)]=high_
    df_final.loc[category, ("se", method)]=se_



def write_stat(df_final, category, kappa_, low_, high_, se_):
    df_final.loc[category, ("kappa score")]=kappa_
    df_final.loc[category, ("ci low")]=low_
    df_final.loc[category, ("ci high")]=high_
    df_final.loc[category, ("se")]=se_


In [8]:

# Enter the path to the tsv file with the rating from the first reviwer
path1_tsv = "../human_rating/rating_90/rating_90_O.csv"

# Read the TSV file into a Pandas DataFrame
df_rating_1 = pd.read_csv(path1_tsv, sep = "\t", index_col=False, header= None)
df_rating_1.fillna('0', inplace=True)


In [9]:
# Enter the path to the tsv file with the rating from the second reviwer
path2_tsv = "../human_rating/rating_90/rating_90_E.csv"

# Read the TSV file into a Pandas DataFrame
df_rating_2 = pd.read_csv(path2_tsv, sep = "\t", index_col=False, header= None)
df_rating_2.fillna('0', inplace=True)

In [10]:
# List of categories you want to make statistics for
list_categories = [
        "Models and algorithms",
        "Datasets",
        "Code",
        "Experimental results",
        "Error bars or statistical significance",
        "Code is or will be available",
        "Statement",
        "Comments",
    ]

In [11]:
# Define a list of statistics to be included in the DataFrame
list_stats = ["kappa score", "ci low", "ci high", "se"]

# Create an index for the rows of the DataFrame
index_line = pd.Index(list_categories + ["Meta-categories", "Repo provided"])

# Create an index for the columns of the DataFrame
index_column = pd.Index(list_stats)

# Create an empty DataFrame with the specified row and column indices
df_final = pd.DataFrame(index=index_line, columns=index_column)


In [12]:

# Loop through each category
for category in range(len(list_categories)):
    
    # Initialize lists to store reviews from both reviewers
    all_reviews_1 = []
    all_reviews_2 = []
    
    # Iterate over the three reviews for the current category
    for i in range(3):
        # Calculate the column index for the current category and review
        column_id = i * 9 + 3 + category
        
        # Extract reviews from both reviewers and concatenate them to the respective lists
        list_review_1 = df_rating_1.loc[2:, column_id].values.tolist()
        list_review_2 = df_rating_2.loc[2:, column_id].values.tolist()

        all_reviews_1 = all_reviews_1 + list_review_1
        all_reviews_2 = all_reviews_2 + list_review_2

    # Calculate the total number of reviews for the current item
    N = len(all_reviews_1)
    
    print(f"For \'{df_rating_1.loc[1, column_id]}\' item (over {N} reviews):")
    
    # Create a confusion matrix to compare the ratings of the two reviewers
    confusion_matrix = create_confusion_matrix(list_1=all_reviews_1, list_2=all_reviews_2)
    
    # Calculate observed and expected proportions
    po_ = observed_proportion(confusion_matrix)
    pe_ = expected_proportion(confusion_matrix)
    
    # Calculate Cohen's kappa
    kappa_ = kappa(po_, pe_)
    
    # If kappa is not 1, perform bootstrap resampling to estimate standard error and confidence interval
    if kappa_ != 1:
        kappa_btp, se_btp, low_btp, high_btp = bootstrap_cqk(y_true=all_reviews_1, y_pred=all_reviews_2)
    else:
        kappa_btp, se_btp, low_btp, high_btp = 1.0, 0.0, 1.0, 1.0
    
    # Write the calculated statistics to the DataFrame
    write_stat(df_final, list_categories[category], kappa_, low_btp, high_btp, se_btp)

    # Print the results
    print(f"Cohen's kappa = {kappa_}")
    print(f"Standard error (bootstrap) = {se_btp}")
    print(f"CI bootstrap = [{low_btp}, {high_btp}]")
    print("**************************************************")

    # ######## For sanity check
    # print(f"kappa cohen sklearn = {kappa_sklearn}")
    # kappa_sklearn = cohen_kappa_score(all_reviews_1, all_reviews_2)
    # df_final.loc[list_categories[category], ("kappa score", "sklearn")]=kappa_sklearn

    # ######## For sanity check
    # print(f"kappa cohen bootstrap = {kappa_btp}")

    # ######## For sanity check
    # data = [all_reviews_1, all_reviews_2]
    # data_T = np.array(data).T
    # data_fleiss_ = aggregate_raters(data_T)
    # kappa_fleiss_ = fleiss_kappa(data_fleiss_[0])
    # df_final.loc[list_categories[category], ("kappa score", "fleiss")]=kappa_fleiss_
    # print(f"kappa fleiss statsmodels = {kappa_fleiss_}")

    # ######## For sanity check
    # print(f"standard error (cohen) = {sd_cohen_ / sqrt(N)}")
    
    # ######## For sanity check
    # sd_cohen_ = sd_cohen(po_, pe_)
    # se_cohen = sd_cohen_ / sqrt(N)
    # low_parametric_cohen=-1.96 * se_cohen + kappa_
    # high_parametric_cohen=1.96 * se_cohen + kappa_
    # generic_write_stat(df_final, list_categories[category], "cohen", kappa_, -1.96 * se_cohen + kappa_, 1.96 * se_cohen + kappa_, se_cohen )

    # print(f"CI parametric from Cohen's SE = [{low_parametric_cohen}, {high_parametric_cohen}]")
 
    

For 'Models and algorithms' item (over 270 reviews):
Cohen's kappa = 0.7544080604534007
Standard error (bootstrap) = 0.045031294422793584
CI bootstrap = [0.6698617578457211, 0.8384990152953175]
**************************************************
For 'Datasets' item (over 270 reviews):
Cohen's kappa = 0.9085872576177285
Standard error (bootstrap) = 0.028239424567769098
CI bootstrap = [0.8497683972990939, 0.9574865721626991]
**************************************************
For 'Code' item (over 270 reviews):
Cohen's kappa = 0.9107142857142856
Standard error (bootstrap) = 0.024555002630953508
CI bootstrap = [0.8591630462989279, 0.9555050175742116]
**************************************************
For 'Experimental results' item (over 270 reviews):
Cohen's kappa = 0.8637248539909151
Standard error (bootstrap) = 0.03525878137696296
CI bootstrap = [0.7926259695574062, 0.9261772943473944]
**************************************************
For 'Error bars or statistical significance' item (o

In [13]:
# Initialize lists to store reviews from both reviewers for meta-categories
list_meta_1 = []
list_meta_2 = []

# Iterate over the three reviews for meta-categories
for i in range(3):
    # Calculate the column index for the current meta-category and review
    column_id = i + 29
    
    # Extract reviews from both reviewers and concatenate them to the respective lists
    list_review_1 = df_rating_1.loc[2:, column_id].values.tolist()
    list_review_2 = df_rating_2.loc[2:, column_id].values.tolist()

    list_meta_1 = list_meta_1 + list_review_1
    list_meta_2 = list_meta_2 + list_review_2

# Count the occurrences of "Unusable (meta)" in both lists
test = list_meta_1.count("Unusable (meta)")
test2 = list_meta_2.count("Unusable (meta)")

# Calculate the total number of reviews for meta-categories
N = len(list_meta_1)

# Create a confusion matrix to compare the ratings of the two reviewers for meta-categories
confusion_matrix = create_confusion_matrix(list_1=list_meta_1, list_2=list_meta_2)

# Calculate observed and expected proportions for meta-categories
po_ = observed_proportion(confusion_matrix)
pe_ = expected_proportion(confusion_matrix)

# Calculate Cohen's kappa for meta-categories
kappa_ = kappa(po_, pe_)

# If kappa is not 1, perform bootstrap resampling to estimate standard error and confidence interval
if kappa_ != 1:
    kappa_btp, se_btp, low_btp, high_btp = bootstrap_cqk(y_true=all_reviews_1, y_pred=all_reviews_2)
else:
    kappa_btp, se_btp, low_btp, high_btp = 1.0, 0.0, 1.0, 1.0

# Write the calculated statistics to the DataFrame for meta-categories
write_stat(df_final, "Meta-categories", kappa_, low_btp, high_btp, se_btp)

# Print the results for meta-categories
print(f"For \'Meta-category\' item (over {N} reviews):")

print(f"Cohen's kappa = {kappa_}")
print(f"Standard error (bootstrap) = {se_btp}")
print(f"CI bootstrap = [{low_btp}, {high_btp}]")
print("**************************************************")

# ######## For sanity check
# print(f"We can count {test} reviews unusable for the first rater and {test2} reviews unusable for the second.")

# ######## For sanity check
# kappa_sklearn = cohen_kappa_score(all_reviews_1, all_reviews_2)
# print(f"kappa cohen sklearn = {kappa_sklearn}")
# kappa_sklearn = cohen_kappa_score(all_reviews_1, all_reviews_2)
# df_final.loc[list_categories[category], ("kappa score", "sklearn")]=kappa_sklearn

# ######## For sanity check
# print(f"kappa cohen bootstrap = {kappa_btp}")

# ######## For sanity check
# data = [all_reviews_1, all_reviews_2]
# data_T = np.array(data).T
# data_fleiss_ = aggregate_raters(data_T)
# kappa_fleiss_ = fleiss_kappa(data_fleiss_[0])
# df_final.loc[list_categories[category], ("kappa score", "fleiss")]=kappa_fleiss_
# print(f"kappa fleiss statsmodels = {kappa_fleiss_}")

# ######## For sanity check
# sd_cohen_ = sd_cohen(po_, pe_)
# print(f"standard error (cohen) = {sd_cohen_ / sqrt(N)}")

# ######## For sanity check
# se_cohen = sd_cohen_ / sqrt(N)
# low_parametric_cohen=-1.96 * se_cohen + kappa_
# high_parametric_cohen=1.96 * se_cohen + kappa_
# write_stat(df_final, list_categories[category], "cohen", kappa_, -1.96 * se_cohen + kappa_, 1.96 * se_cohen + kappa_, se_cohen )

# print(f"CI parametric from Cohen's SE = [{low_parametric_cohen}, {high_parametric_cohen}]")

For 'Meta-category' item (over 270 reviews):
Cohen's kappa = 0.8014027898179524
Standard error (bootstrap) = 0.02797923379338488
CI bootstrap = [0.76181813602006, 0.8667981355111253]
**************************************************


In [14]:
# Add repo provided review

# Extract values from specific columns in dataframes and convert to lists
list_repo_1 = df_rating_1.loc[2:, 39].values.tolist()
list_repo_2 = df_rating_2.loc[2:, 39].values.tolist()

# Get the length of the lists
N = len(list_repo_1)

# Create confusion matrix based on the provided lists
confusion_matrix = create_confusion_matrix(list_1=list_repo_1, list_2=list_repo_2)

# Calculate observed and expected proportions, and Cohen's kappa
po_ = observed_proportion(confusion_matrix)
pe_ = expected_proportion(confusion_matrix)
kappa_ = kappa(po_, pe_)

# Check if kappa is not equal to 1, perform bootstrap to estimate confidence intervals
if (kappa_ != 1):
    kappa_btp, se_btp, low_btp, high_btp = bootstrap_cqk(y_true=all_reviews_1, y_pred=all_reviews_2)
else:
    # Set values to 1.0 for cases where kappa is 1
    kappa_btp, se_btp, low_btp, high_btp = 1.0, 0.0, 1.0, 1.0

# Write statistical results to a dataframe
write_stat(df_final, "Repo provided", kappa_, low_btp, high_btp, se_btp)

# Print results for 'Repo provided' item
print(f"For 'Repo provided' item (over {N} reviews):")
print(f"Cohen's kappa = {kappa_}")
print(f"Standard error (bootstrap) = {se_btp}")
print(f"CI bootstrap = [{low_btp}, {high_btp}]")
print("**************************************************")


######### For sanity check
# print(f"We can count {test} reviews unusable for the first rater and {test2} reviews unusable for the second.")

######### For sanity check
# print(f"kappa cohen sklearn = {kappa_sklearn}")
# kappa_sklearn = cohen_kappa_score(all_reviews_1, all_reviews_2)
# df_final.loc[list_categories[category], ("kappa score", "sklearn")]=kappa_sklearn

######### For sanity check
# print(f"kappa cohen bootstrap = {kappa_btp}")

######### For sanity check
# data = [all_reviews_1, all_reviews_2]
# data_T = np.array(data).T
# data_fleiss_ = aggregate_raters(data_T)
# kappa_fleiss_ = fleiss_kappa(data_fleiss_[0])
# df_final.loc[list_categories[category], ("kappa score", "fleiss")]=kappa_fleiss_
# print(f"kappa fleiss statsmodels = {kappa_fleiss_}")

######### For sanity check
# print(f"standard error (cohen) = {sd_cohen_ / sqrt(N)}")

######### For sanity check
# sd_cohen_ = sd_cohen(po_, pe_)
# se_cohen = sd_cohen_ / sqrt(N)
# low_parametric_cohen=-1.96 * se_cohen + kappa_
# high_parametric_cohen=1.96 * se_cohen + kappa_
#write_stat(df_final, list_categories[category], "cohen", kappa_, -1.96 * se_cohen + kappa_, 1.96 * se_cohen + kappa_, se_cohen )

#print(f"CI parametric from Cohen's SE = [{low_parametric_cohen}, {high_parametric_cohen}]")


For 'Repo provided' item (over 90 reviews):
Cohen's kappa = 1.0
Standard error (bootstrap) = 0.0
CI bootstrap = [1.0, 1.0]
**************************************************


In [15]:
# Save final dataframe to a CSV file

# Define the output directory path
output_directory = Path(f"../miccai2023/stats_inter_rater")

# Check if the output directory exists, and create it if it doesn't
if not output_directory.is_dir():
    os.mkdir(output_directory)

# Define the path for the inter-rater statistics file within the output directory
path_inter_rater_stats = output_directory / 'inter_rater_stats.csv'

# Save the final dataframe to the CSV file
df_final.to_csv(path_inter_rater_stats, index=True, sep="\t", encoding='utf-8')


In [16]:
# Create a dictionary to store metadata information
data = {}

# Calculate and store the number of papers (assuming papers are represented in all_reviews_1)
data["nb_papers"] = len(all_reviews_1) / 3  # Assuming each paper has 3 reviews

# Store the total number of reviews
data["nb_reviews"] = len(all_reviews_1)

# Store the current date and time
data["date"] = str(datetime.date.today())
data["time"] = str(datetime.datetime.utcnow())

# Store the file paths of the input data
data["data_path1"] = str(path1_tsv)
data["data_path2"] = str(path2_tsv)

# Convert the dictionary to a JSON-formatted string with indentation for readability
json_data = json.dumps(data, skipkeys=True, indent=4)

# Define the path for the JSON file within the output directory
json_path = output_directory / "data.json"

# Write the JSON data to the file
with open(json_path, "w") as f:
    f.write(json_data)


In [17]:
from tabulate import tabulate
print(tabulate(df_final, headers='keys', tablefmt='psql', floatfmt=".2f"))

+----------------------------------------+---------------+----------+-----------+------+
|                                        |   kappa score |   ci low |   ci high |   se |
|----------------------------------------+---------------+----------+-----------+------|
| Models and algorithms                  |          0.75 |     0.67 |      0.84 | 0.05 |
| Datasets                               |          0.91 |     0.85 |      0.96 | 0.03 |
| Code                                   |          0.91 |     0.86 |      0.96 | 0.02 |
| Experimental results                   |          0.86 |     0.79 |      0.93 | 0.04 |
| Error bars or statistical significance |          1.00 |     1.00 |      1.00 | 0.00 |
| Code is or will be available           |          0.89 |     0.81 |      0.95 | 0.03 |
| Statement                              |          0.74 |     0.67 |      0.80 | 0.03 |
| Comments                               |          0.82 |     0.76 |      0.87 | 0.03 |
| Meta-categories    